# Linear Regression Widget

In the last chapter, we iterated through combinations of coefficients to calculate their $R^2$ value. In this chapter, a simple web application is built that allows you to choose the feature you want to predict housing price and the possible coefficients. The application is built with a dropdown menu and two sliders provided from [Jupyter Widgets][1].

[1]: https://ipywidgets.readthedocs.io/en/stable

In [ ]:
from ipywidgets import interact, Dropdown, FloatSlider
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
housing = pd.read_csv('../data/housing_sample.csv')
housing.head()

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots(figsize=(8, 5))
cols = ['LotFrontage', 'GrLivArea', 'GarageArea']
dd = Dropdown(options=cols, value='GrLivArea')
w0_slider = FloatSlider()
w1_slider = FloatSlider()

def scatter(feature):
    ax.clear()
    hs = housing.sort_values(feature).dropna()
    hs.plot(x=feature, y='SalePrice', kind='scatter', ax=ax)
    
    hs_head = hs.head(100)
    hs_tail = hs.tail(100)
    y2, y1 = hs_head['SalePrice'].values, hs_tail['SalePrice'].values
    x2, x1 = hs_head[feature].values, hs_tail[feature].values
    slopes = (y2 - y1) / (x2 - x1)
    intercepts = (-slopes * x2 + y2)
    
    med_intercepts = np.nanmedian(intercepts)
    std_intercepts = np.nanstd(intercepts)
    med_slopes = np.nanmedian(slopes)
    std_slopes = np.nanstd(slopes)
    
    w0_slider.set_trait('value', med_intercepts)
    w0_slider.set_trait('min', med_intercepts - 3 * std_intercepts)
    w0_slider.set_trait('max', med_intercepts + 3 * std_intercepts)
    w1_slider.set_trait('value', med_slopes)
    w1_slider.set_trait('min', med_slopes - 3 * std_slopes)
    w1_slider.set_trait('max', med_slopes + 3 * std_slopes)
    
def pick_w(w0, w1):
    for line in ax.lines:
        line.remove()
    col = dd.get_interact_value()
    x = housing[col]
    y = housing['SalePrice']
    x_plot = x.agg(['min', 'max']).values
    ax.plot(x_plot, w0 + w1 * x_plot, color='red')
    ax.set_ylim(y.min() * -1.1, y.max() * 1.1)
    y_pred = w0 + w1 * x
    sse = ((y - y_pred) ** 2).sum()
    variance = ((y - y.mean()) ** 2).sum()
    r2 = (variance - sse) / variance
    ax.set_title(f'$R^2$ = {r2:.2f}')
    
interact(scatter, feature=dd)
interact(pick_w, w0=w0_slider, w1=w1_slider)